In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data=pd.read_csv("/Users/robertmarks/Desktop/kaggle/spaceship_titanic/input/train.csv")

In [3]:
data

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [7]:
#investigate data

In [9]:
data.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [11]:
nan_rows = data[data.isna().any(axis=1)]
nan_counts = data.isna().sum()

print(nan_counts)

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64


To deal with the Nan's let's try to fill them with meaningful things
Let's start with the HomePlanet, there are 201 missing; let's check if we can infer some based on the cabin (matching cabin means most probably same start)
 if not we can also check for last name; if last name is the same as someone elses they probably are related and family is most often close

In [14]:
#deal with last Name
data['Last_Name'] = data['Name'].str.split(' ').str[-1].fillna('Nobody')

In [16]:
def fill_missing_homeplanet(df):
    # Create a copy of the dataframe to avoid modifying the original
    df = df.copy()
    
    # Counter for tracking assignments and remaining NaN values
    initial_missing = df['HomePlanet'].isna().sum()
    print(f"Initial number of missing HomePlanet values: {initial_missing}")
    
    # Step 1: Fill missing HomePlanet based on Last_Name matches
    # First, create a dictionary of known Last_Name to HomePlanet mappings
    # (excluding 'Nobody' and NaN values)
    known_last_name_mapping = df[
        (df['Last_Name'] != 'Nobody') & 
        (~df['Last_Name'].isna()) & 
        (~df['HomePlanet'].isna())
    ].groupby('Last_Name')['HomePlanet'].first().to_dict()
    
    # Function to fill HomePlanet based on Last_Name
    def fill_by_last_name(row):
        if pd.isna(row['HomePlanet']) and not pd.isna(row['Last_Name']):
            if row['Last_Name'] != 'Nobody':
                return known_last_name_mapping.get(row['Last_Name'])
        return row['HomePlanet']
    
    # Apply the Last_Name based filling
    df['HomePlanet'] = df.apply(fill_by_last_name, axis=1)
    
    # Count how many were filled by Last_Name
    after_last_name = df['HomePlanet'].isna().sum()
    filled_by_last_name = initial_missing - after_last_name
    print(f"Filled by Last_Name: {filled_by_last_name}")
    
    # Step 2: Fill remaining missing HomePlanet based on Cabin matches
    # Create a dictionary of known Cabin to HomePlanet mappings
    known_cabin_mapping = df[
        (~df['Cabin'].isna()) & 
        (~df['HomePlanet'].isna())
    ].groupby('Cabin')['HomePlanet'].first().to_dict()
    
    # Function to fill HomePlanet based on Cabin
    def fill_by_cabin(row):
        if pd.isna(row['HomePlanet']) and not pd.isna(row['Cabin']):
            return known_cabin_mapping.get(row['Cabin'])
        return row['HomePlanet']
    
    # Apply the Cabin based filling
    df['HomePlanet'] = df.apply(fill_by_cabin, axis=1)
    
    # Count how many were filled by Cabin
    final_missing = df['HomePlanet'].isna().sum()
    filled_by_cabin = after_last_name - final_missing
    print(f"Filled by Cabin: {filled_by_cabin}")
    print(f"Remaining missing HomePlanet values: {final_missing}")
    
    # Step 3: Fill remaining missing HomePlanet with the most common home planet
    if final_missing > 0:
        most_common_homeplanet = df['HomePlanet'].mode()[0]
        df['HomePlanet'].fillna(most_common_homeplanet, inplace=True)
        print(f"Filled by most common HomePlanet: {final_missing}")
        final_missing = df['HomePlanet'].isna().sum()

    print("Missing home planets", final_missing)
    
    return df

In [18]:
df= fill_missing_homeplanet(data)

Initial number of missing HomePlanet values: 201
Filled by Last_Name: 186
Filled by Cabin: 2
Remaining missing HomePlanet values: 13
Filled by most common HomePlanet: 13
Missing home planets 0


/var/folders/gm/ptz_rd3s5fv5m6lks9m633wr0000gn/T/ipykernel_75171/3480501072.py:58: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['HomePlanet'].fillna(most_common_homeplanet, inplace=True)


In [20]:
def infer_cabin_by_lastname(df):
    # Create a dictionary of known LastName to Cabin mappings (excluding "Nobody" and NaN values)
    known_lastname_mapping = df[
        (df['Last_Name'] != 'Nobody') & 
        (~df['Last_Name'].isna()) & 
        (~df['Cabin'].isna())
    ].groupby('Last_Name')['Cabin'].first().to_dict()
    
    # Function to fill Cabin based on LastName
    def fill_by_lastname(row):
        if pd.isna(row['Cabin']) and row['Last_Name'] != 'Nobody':
            return known_lastname_mapping.get(row['Last_Name'])
        return row['Cabin']
    
    # Apply the LastName-based filling
    df['Cabin'] = df.apply(fill_by_lastname, axis=1)
    return df

In [22]:
# Apply Step 1: Infer Cabin by Last Name
df = infer_cabin_by_lastname(df)

In [26]:
#let's try to replace the cabin
# Step 1: Split the Cabin column into cabin_deck, cabin_num, and cabin_side
df[['cabin_deck', 'cabin_num', 'cabin_side']] = df['Cabin'].str.split('/', expand=True)

In [28]:
#for people that are in CryoSleep -> if not spent any money in CryoSleep, if spent money on something, not in CryoSleep if one nan leave nan
# Define the spending columns
# Function to infer CryoSleep
def infer_cryosleep(row):
    spending_columns = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    if any(pd.notna(row[col]) and row[col] > 0 for col in spending_columns):  # If any spending > 0
        return False
    elif all(row[col] == 0 for col in spending_columns):  # If all spending == 0
        return True
    else:  # If any spending is NaN
        nan_count = sum(pd.isna(row[col]) for col in spending_columns)
        if nan_count>1:
            return False
        else:
            return True

# Apply the function to infer CryoSleep
df['CryoSleep'] = df.apply(infer_cryosleep, axis=1)


In [30]:
nan_rows = df[df.isna().any(axis=1)]
nan_counts = df.isna().sum()

print(nan_counts)

PassengerId       0
HomePlanet        0
CryoSleep         0
Cabin            11
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
Last_Name         0
cabin_deck       11
cabin_num        11
cabin_side       11
dtype: int64


In [32]:
# Define the service columns
service_columns = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

# Step 3: Infer Cabin Deck from Spending Patterns
def infer_cabin_deck_from_spending(df):
    # Calculate the median spending for each cabin deck
    median_spending_by_deck = df.groupby('cabin_deck')[service_columns].median()
    
    # Function to find the most similar deck based on spending
    def find_most_similar_deck(row):
        if pd.isna(row['cabin_deck']):
            # Calculate the Euclidean distance between the passenger's spending and the median spending of each deck
            distances = {}
            for deck, median_spending in median_spending_by_deck.iterrows():
                distance = np.linalg.norm(row[service_columns].fillna(0) - median_spending.fillna(0))
                distances[deck] = distance
            # Assign the deck with the smallest distance
            return min(distances, key=distances.get)
        return row['cabin_deck']
    
    # Apply the spending-based filling
    df['cabin_deck'] = df.apply(find_most_similar_deck, axis=1)
    return df


# Infer Cabin Deck from Spending Patterns
df = infer_cabin_deck_from_spending(df)


/var/folders/gm/ptz_rd3s5fv5m6lks9m633wr0000gn/T/ipykernel_75171/2703569800.py:15: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  distance = np.linalg.norm(row[service_columns].fillna(0) - median_spending.fillna(0))
/var/folders/gm/ptz_rd3s5fv5m6lks9m633wr0000gn/T/ipykernel_75171/2703569800.py:15: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  distance = np.linalg.norm(row[service_columns].fillna(0) - median_spending.fillna(0))
/var/folders/gm/ptz_rd3s5fv5m6lks9m633wr0000gn/T/ipykernel_75171/2703569800.py:15: FutureWarning: Downcasting object dtype arrays 

In [34]:
#assign random number to cabin's depending on the distribution of the deck; 
def assign_random_cabin_num(df):
    # Ensure cabin_num is numeric
    df['cabin_num'] = pd.to_numeric(df['cabin_num'], errors='coerce')
    
    # Group by cabin_deck and calculate the min and max cabin_num for each deck
    deck_stats = df.groupby('cabin_deck')['cabin_num'].agg(['min', 'max']).reset_index()
    
    # Merge the stats back into the original DataFrame
    df = df.merge(deck_stats, on='cabin_deck', how='left')
    
    # Assign a random cabin number within the range of the deck
    def random_cabin_num(row):
        if pd.isna(row['cabin_num']):  # Only fill missing cabin_num
            return np.random.randint(row['min'], row['max'] + 1)
        return row['cabin_num']
    
    df['cabin_num'] = df.apply(random_cabin_num, axis=1)
    
    # Drop the temporary min and max columns
    df = df.drop(columns=['min', 'max'])
    
    return df


In [36]:
# assign minority side to the cabin depending on the level, so that the cabin's are equally distributed on a level
def assign_minority_side(df):
    # Group by cabin_deck and cabin_side to count the number of cabins on each side
    side_counts = df.groupby(['cabin_deck', 'cabin_side']).size().unstack(fill_value=0)
    
    # Function to assign the minority side for a given deck
    def get_minority_side(deck):
        if deck not in side_counts.index:
            return np.random.choice(['S', 'P'])  # If deck is new, randomly assign a side
        return 'S' if side_counts.loc[deck, 'S'] < side_counts.loc[deck, 'P'] else 'P'
    
    # Assign the minority side to cabins with missing cabin_side
    df['cabin_side'] = df.apply(
        lambda row: get_minority_side(row['cabin_deck']) if pd.isna(row['cabin_side']) else row['cabin_side'],
        axis=1
    )
    
    return df

In [38]:
# Apply Function 1: Assign random cabin numbers based on deck distribution
df = assign_random_cabin_num(df)

# Apply Function 2: Assign minority side to ensure equal distribution
df = assign_minority_side(df)

In [40]:
#Let's investigate the missing spendings

In [42]:
# Define the service columns
service_columns = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

# Step 1: Identify passengers with missing spending values
df['missing_count'] = df[service_columns].isna().sum(axis=1)

# Step 2: Count how many passengers have missing values for each number of services
missing_distribution = df['missing_count'].value_counts().sort_index()

# Step 3: Analyze the distribution of missing values
print("Distribution of Missing Values per Passenger:")
print(missing_distribution)

# Step 4: Identify passengers missing all services
passengers_missing_all = df[df['missing_count'] == len(service_columns)]['PassengerId']
print("\nPassengers Missing All Services:")
print(passengers_missing_all.tolist())

Distribution of Missing Values per Passenger:
missing_count
0    7785
1     873
2      35
Name: count, dtype: int64

Passengers Missing All Services:
[]


In [44]:
#let's deal with the spendings
#1st let's follow our argumentation/ intuition from above -> if someone is in CryoSleep, they don't spent any money
df.loc[df['CryoSleep'] == True, service_columns] = df.loc[df['CryoSleep'] == True, service_columns].fillna(0)
#2rd let's infer the missing values if only one is missing -> impute a randomNumber between the highest and lowest spending of this person of the other services
def impute_single_missing_spending(row):
    # Count the number of missing values in service_columns
    missing_count = row[service_columns].isna().sum()
    
    # If only one service is missing, impute a random value between the lowest and highest spending of other services
    if missing_count == 1:
        # Get the non-missing spending values
        non_missing_spendings = row[service_columns].dropna()
        
        # Calculate the range for imputation
        min_spending = non_missing_spendings.min()
        max_spending = non_missing_spendings.max()
        
        # Impute a random value between min and max
        for service in service_columns:
            if pd.isna(row[service]):
                row[service] = np.random.uniform(min_spending, max_spending)
    
    return row

df = df.apply(impute_single_missing_spending, axis=1)
#3rd if they have more than one missing, Group by relevant columns and calculate the median spending for each service
group_stats = df.groupby(['HomePlanet', 'Destination', 'cabin_deck'])[service_columns].median()

# Fill missing values based on group statistics
for service in service_columns:
    df[service] = df.apply(
        lambda row: group_stats.loc[(row['HomePlanet'], row['Destination'], row['cabin_deck']), service] 
        if pd.isna(row[service]) else row[service],
        axis=1
    )

In [47]:
#let's deal with destination
# Define the unique destinations and home planets
destinations = ['TRAPPIST-1e', 'PSO J318.5-22', '55 Cancri e']
home_planets = ['Europa', 'Earth', 'Mars']

# Step 1: Infer Destination based on Last_Name
def infer_destination_by_lastname(df):
    # Create a dictionary of known Last_Name to Destination mappings (excluding "Nobody" and NaN values)
    known_lastname_mapping = df[
        (df['Last_Name'] != 'Nobody') & 
        (~df['Last_Name'].isna()) & 
        (~df['Destination'].isna())
    ].groupby('Last_Name')['Destination'].first().to_dict()
    
    # Function to fill Destination based on Last_Name
    def fill_by_lastname(row):
        if pd.isna(row['Destination']) and row['Last_Name'] != 'Nobody':
            return known_lastname_mapping.get(row['Last_Name'])
        return row['Destination']
    
    # Apply the Last_Name-based filling
    df['Destination'] = df.apply(fill_by_lastname, axis=1)
    return df

# Step 2: Infer Destination based on Home Planet probabilities
def infer_destination_by_homeplanet(df):
    # Calculate the probability distribution of Destination for each HomePlanet
    destination_prob = df.groupby('HomePlanet')['Destination'].value_counts(normalize=True).unstack()
    
    # Function to fill Destination based on HomePlanet probabilities
    def fill_by_homeplanet(row):
        if pd.isna(row['Destination']):
            probabilities = destination_prob.loc[row['HomePlanet']]
            return np.random.choice(probabilities.index, p=probabilities.values)
        return row['Destination']
    
    # Apply the HomePlanet-based filling
    df['Destination'] = df.apply(fill_by_homeplanet, axis=1)
    return df

# Apply Step 1: Infer Destination by Last_Name
df = infer_destination_by_lastname(df)

# Apply Step 2: Infer Destination by HomePlanet probabilities
df = infer_destination_by_homeplanet(df)

In [50]:
# Define the service columns
service_columns = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

# Step 1: Compare mean and median spending for VIP and non-VIP passengers
vip_spending = df[df['VIP'] == True][service_columns]
non_vip_spending = df[df['VIP'] == False][service_columns]

# Calculate mean and median spending for VIP and non-VIP passengers
vip_mean = vip_spending.mean()
vip_median = vip_spending.median()
non_vip_mean = non_vip_spending.mean()
non_vip_median = non_vip_spending.median()

print("Mean Spending for VIP Passengers:")
print(vip_mean)
print("\nMedian Spending for VIP Passengers:")
print(vip_median)

print("\nMean Spending for Non-VIP Passengers:")
print(non_vip_mean)
print("\nMedian Spending for Non-VIP Passengers:")
print(non_vip_median)

# Step 2: Compare overall spending (sum of all services) for VIP and non-VIP passengers
df['total_spending'] = df[service_columns].sum(axis=1)

vip_total_spending = df[df['VIP'] == True]['total_spending']
non_vip_total_spending = df[df['VIP'] == False]['total_spending']

# Calculate mean and median total spending for VIP and non-VIP passengers
vip_total_mean = vip_total_spending.mean()
vip_total_median = vip_total_spending.median()
non_vip_total_mean = non_vip_total_spending.mean()
non_vip_total_median = non_vip_total_spending.median()

print("\nMean Total Spending for VIP Passengers:")
print(vip_total_mean)
print("\nMedian Total Spending for VIP Passengers:")
print(vip_total_median)

print("\nMean Total Spending for Non-VIP Passengers:")
print(non_vip_total_mean)
print("\nMedian Total Spending for Non-VIP Passengers:")
print(non_vip_total_median)

Mean Spending for VIP Passengers:
RoomService      482.052335
FoodCourt       1770.785551
ShoppingMall     265.240717
Spa              761.700518
VRDeck          1219.889440
dtype: float64

Median Spending for VIP Passengers:
RoomService       0.000000
FoodCourt       290.283552
ShoppingMall      0.000000
Spa              39.000000
VRDeck           31.000000
dtype: float64

Mean Spending for Non-VIP Passengers:
RoomService     222.780762
FoodCourt       425.474758
ShoppingMall    179.280753
Spa             303.361698
VRDeck          282.690736
dtype: float64

Median Spending for Non-VIP Passengers:
RoomService     0.0
FoodCourt       0.0
ShoppingMall    0.0
Spa             0.0
VRDeck          0.0
dtype: float64

Mean Total Spending for VIP Passengers:
4499.668560502835

Median Total Spending for VIP Passengers:
2785.0

Mean Total Spending for Non-VIP Passengers:
1413.588706725356

Median Total Spending for Non-VIP Passengers:
716.0


In [52]:
#Infer VIP Status from KNN

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Define the service columns and features
service_columns = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
features = ['cabin_deck'] + service_columns

# Separate the data into training and test sets
train_df = df[df['VIP'].notna()]  # Rows with known VIP status
test_df = df[df['VIP'].isna()]    # Rows with missing VIP status

# Prepare the features and target
X_train = train_df[features]
y_train = train_df['VIP']
X_test = test_df[features]

# Preprocessing pipeline
# One-hot encode 'cabin_deck' and scale numerical features
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), ['cabin_deck']),  # One-hot encode 'cabin_deck'
        ('num', StandardScaler(), service_columns)  # Scale spending columns
    ]
)

# Create a pipeline with preprocessing and KNN classifier
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', KNeighborsClassifier(n_neighbors=3))  # Use 3 nearest neighbors
])

y_train = y_train.astype('category')  # Ensure categorical dtype

# Train the model
pipeline.fit(X_train, y_train)

# Predict missing VIP labels
test_df['VIP'] = pipeline.predict(X_test)

# Update the original DataFrame with the predicted VIP labels
df.loc[test_df.index, 'VIP'] = test_df['VIP']

/var/folders/gm/ptz_rd3s5fv5m6lks9m633wr0000gn/T/ipykernel_75171/2007379956.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['VIP'] = pipeline.predict(X_test)


In [54]:
#Let's impute the missing ages "simply" with the median value
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')  # Use median for imputation
df[['Age']] = imputer.fit_transform(df[['Age']])

In [56]:
nan_rows = df[df.isna().any(axis=1)]
nan_counts = df.isna().sum()

print(nan_counts)

PassengerId         0
HomePlanet          0
CryoSleep           0
Cabin              11
Destination         0
Age                 0
VIP                 0
RoomService         0
FoodCourt           0
ShoppingMall        0
Spa                 0
VRDeck              0
Name              200
Transported         0
Last_Name           0
cabin_deck          0
cabin_num           0
cabin_side          0
missing_count       0
total_spending      0
dtype: int64


In [58]:
#Feature engineering!

In [60]:
#let's add a column for has spent money and family and family_size
df['money_spent']=df['total_spending']>0

In [62]:
# Step 1: Exclude rows where Last_Name is "Nobody" for family mapping
df_family = df[df['Last_Name'] != 'Nobody'].copy()

# Step 2: Create a unique family identifier based on Last_Name and Cabin
df_family['family_id'] = df_family.groupby(['Last_Name', 'Cabin']).ngroup()

# Step 3: Calculate family size
family_size = df_family.groupby('family_id')['PassengerId'].transform('count')


# Step 4: Map family_size and Is_family back to the original DataFrame
df = df.merge(
    df_family[['PassengerId', 'family_id']].assign(family_size=family_size),
    on='PassengerId',
    how='left'
)

# Step 5: Fill NaN values for family_size (for passengers with Last_Name "Nobody")
df['family_size'] = df['family_size'].fillna(0).astype(int)

# Step 6: Create the Is_family column
df['Is_family'] = df['family_size'] > 1

# Drop the temporary family_id column (optional)
df = df.drop(columns=['family_id'])

     PassengerId HomePlanet  CryoSleep     Cabin    Destination   Age    VIP  \
0        0001_01     Europa       True     B/0/P    TRAPPIST-1e  39.0  False   
1        0002_01      Earth      False     F/0/S    TRAPPIST-1e  24.0  False   
2        0003_01     Europa      False     A/0/S    TRAPPIST-1e  58.0   True   
3        0003_02     Europa      False     A/0/S    TRAPPIST-1e  33.0  False   
4        0004_01      Earth      False     F/1/S    TRAPPIST-1e  16.0  False   
...          ...        ...        ...       ...            ...   ...    ...   
8688     9276_01     Europa      False    A/98/P    55 Cancri e  41.0   True   
8689     9278_01      Earth       True  G/1499/S  PSO J318.5-22  18.0  False   
8690     9279_01      Earth      False  G/1500/S    TRAPPIST-1e  26.0  False   
8691     9280_01     Europa      False   E/608/S    55 Cancri e  32.0  False   
8692     9280_02     Europa      False   E/608/S    TRAPPIST-1e  44.0  False   

      RoomService  FoodCourt  ShoppingM

In [67]:
#Experiment (might be needed to remove) -> is female: most female first names end on a "vocal"! Otw 50/50
def assign_gender(df):
    """
    Extends the DataFrame by adding a 'Gender' column based on the first name.
    - Assigns 'Female' if the first name ends in a vowel ('a', 'e', 'i', 'o', 'u').
    - Assigns 'Male' otherwise.
    - If the first name is missing, assigns gender randomly (50/50 chance).
    
    Parameters:
        df (pd.DataFrame): The input DataFrame containing a 'Name' column.
        
    Returns:
        pd.DataFrame: The modified DataFrame with a new 'Gender' column.
    """
    def determine_gender(name):
        if pd.isna(name) or not isinstance(name, str) or name.strip() == "":
            return np.random.choice(["Male", "Female"])  # 50/50 random assignment
        
        first_name = name.split()[0]  # Extract first name
        return "Female" if first_name[-1].lower() in "aeiou" else "Male"

    df['Gender'] = df['Name'].apply(determine_gender)  # Apply function to each row
    return df

In [69]:
df = assign_gender(df)

In [75]:
df.iloc[0:2]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Last_Name,cabin_deck,cabin_num,cabin_side,missing_count,total_spending,money_spent,family_size,Is_family,Gender
0,0001_01,Europa,True,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,...,Ofracculy,B,0.0,P,0,0.0,False,1,False,Male
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,...,Vines,F,0.0,S,0,736.0,True,1,False,Female


In [80]:
df.keys()

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported', 'Last_Name', 'cabin_deck', 'cabin_num',
       'cabin_side', 'missing_count', 'total_spending', 'money_spent',
       'family_size', 'Is_family', 'Gender'],
      dtype='object')

In [96]:
df['Destination'].unique()

array(['TRAPPIST-1e', 'PSO J318.5-22', '55 Cancri e'], dtype=object)

In [82]:
df.iloc[0]

PassengerId               0001_01
HomePlanet                 Europa
CryoSleep                    True
Cabin                       B/0/P
Destination           TRAPPIST-1e
Age                          39.0
VIP                         False
RoomService                   0.0
FoodCourt                     0.0
ShoppingMall                  0.0
Spa                           0.0
VRDeck                        0.0
Name              Maham Ofracculy
Transported                 False
Last_Name               Ofracculy
cabin_deck                      B
cabin_num                     0.0
cabin_side                      P
missing_count                   0
total_spending                0.0
money_spent                 False
family_size                     1
Is_family                   False
Gender                       Male
Name: 0, dtype: object

In [ ]:
from scipy.stats import chi2_contingency

# Ensure cabin_num is numeric
df['cabin_num'] = pd.to_numeric(df['cabin_num'], errors='coerce')

# Step 1: Create new columns for even/odd and high/low cabin numbers
df['cabin_even_odd'] = np.where(df['cabin_num'] % 2 == 0, 'Even', 'Odd')  # Even or Odd
df['cabin_high_low'] = np.where(df['cabin_num'] > df['cabin_num'].median(), 'High', 'Low')  # High or Low

# Step 2: Analyze the relationship per deck
decks = sorted(df['cabin_deck'].dropna().unique())  # Get unique decks (A to T)

for deck in decks:
    print(f"\nAnalyzing Deck {deck}:")
    deck_df = df[df['cabin_deck'] == deck]  # Filter data for the current deck
    
    # Analyze even/odd vs cabin_side
    contingency_table_even_odd = pd.crosstab(deck_df['cabin_side'], deck_df['cabin_even_odd'])
    print(f"\nContingency Table for Deck {deck} (Even/Odd vs Cabin Side):")
    print(contingency_table_even_odd)
    
    chi2, p, dof, expected = chi2_contingency(contingency_table_even_odd)
    print(f"Chi-squared test for Even/Odd vs Cabin Side: p-value = {p:.4f}")
    
    # Analyze high/low vs cabin_side
    contingency_table_high_low = pd.crosstab(deck_df['cabin_side'], deck_df['cabin_high_low'])
    print(f"\nContingency Table for Deck {deck} (High/Low vs Cabin Side):")
    print(contingency_table_high_low)
    
    chi2, p, dof, expected = chi2_contingency(contingency_table_high_low)
    print(f"Chi-squared test for High/Low vs Cabin Side: p-value = {p:.4f}")

In [ ]:
# Step 1: Analyze Family and VIP Status
# Create a family identifier based on Last_Name and Cabin
df['family_id'] = df.groupby(['Last_Name', 'Cabin']).ngroup()

# Group by family_id and check if VIP status is consistent within families
family_vip = df.groupby('family_id')['VIP'].agg(['mean', 'size']).reset_index()
family_vip['vip_consistent'] = family_vip['mean'].apply(lambda x: x == 0 or x == 1)

# Count the number of families with consistent and inconsistent VIP status
vip_consistency_summary = family_vip['vip_consistent'].value_counts()
print("Family and VIP Status Consistency:")
print(vip_consistency_summary)

# Step 2: Analyze CryoSleep and VIP Status
# Count the number of passengers in CryoSleep with VIP status
cryosleep_vip = df[df['CryoSleep'] == True]['VIP'].value_counts()
print("\nCryoSleep and VIP Status:")
print(cryosleep_vip)

In [ ]:
# Step 1: Exclude rows where Last_Name is "Nobody" for family mapping
df_family = df[df['Last_Name'] != 'Nobody'].copy()

# Step 2: Create a unique family identifier based on Last_Name and Cabin
df_family['family_id'] = df_family.groupby(['Last_Name', 'Cabin']).ngroup()

# Step 3: Calculate family size
family_size = df_family.groupby('family_id')['PassengerId'].transform('count')

# Step 4: Map family_size and Is_family back to the original DataFrame
df = df.merge(
    df_family[['PassengerId', 'family_id']].assign(family_size=family_size),
    on='PassengerId',
    how='left'
)

# Step 5: Fill NaN values for family_size (for passengers with Last_Name "Nobody")
df['family_size'] = df['family_size'].fillna(0).astype(int)

# Step 6: Create the Is_family column
df['Is_family'] = df['family_size'] > 1

# Drop the temporary family_id column (optional)
df = df.drop(columns=['family_id'])

# Display the updated DataFrame
print(df)

In [ ]:
sns.set(style="whitegrid")

# 1. Side vs HomePlanet
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x='cabin_side', hue='Is_family', palette='Set2')
plt.title('CabinSide vs CabinDeck')
plt.xlabel('HomePlanet')
plt.ylabel('Count')
plt.legend(title='Cabin_Side', loc='upper right')
plt.show()

In [ ]:
sns.set(style="whitegrid")

# 1. Side vs HomePlanet
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x='cabin_deck', hue='cabin_side', palette='Set2')
plt.title('CabinSide vs CabinDeck')
plt.xlabel('HomePlanet')
plt.ylabel('Count')
plt.legend(title='Cabin_Side', loc='upper right')
plt.show()

In [ ]:
sns.set(style="whitegrid")

# 1. Side vs HomePlanet
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x='HomePlanet', hue='cabin_side', palette='Set2')
plt.title('CabinSide vs HomePlanet')
plt.xlabel('HomePlanet')
plt.ylabel('Count')
plt.legend(title='Cabin_Side', loc='upper right')
plt.show()

In [ ]:
sns.set(style="whitegrid")

# 2. Side vs HomePlanet
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x='HomePlanet', hue='cabin_deck', palette='Set2')
plt.title('CabinDeck vs HomePlanet')
plt.xlabel('HomePlanet')
plt.ylabel('Count')
plt.legend(title='Cabin_Side', loc='upper right')
plt.show()

In [ ]:
sns.set(style="whitegrid")

# 3. Side vs Destination
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x='HomePlanet', hue='Destination', palette='Set2')
plt.title('CabinSide vs Destination')
plt.xlabel('Destination')
plt.ylabel('Count')
plt.legend(title='Cabin_Side', loc='upper right')
plt.show()

In [ ]:
df['HomePlanet'].unique()

In [ ]:
sns.set(style="whitegrid")

# 4. Side vs Destination
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x='Destination', hue='cabin_deck', palette='Set2')
plt.title('CabinDeck vs Destination')
plt.xlabel('Destination')
plt.ylabel('Count')
plt.legend(title='Cabin_Deck', loc='upper right')
plt.show()

In [ ]:
sns.set(style="whitegrid")

# 4. Side vs Destination
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x='AgeGroup', hue='cabin_side', palette='Set2')
plt.title('CabinDeck vs Destination')
plt.xlabel('Destination')
plt.ylabel('Count')
plt.legend(title='Cabin_Deck', loc='upper right')
plt.show()

In [ ]:
# Convert cabin_num to numeric (handling errors by coercing invalid values to NaN)
df['cabin_num'] = pd.to_numeric(df['cabin_num'], errors='coerce')

# Detailed analysis of cabin_num
print("Summary Statistics for cabin_num:")
print(df['cabin_num'].describe())

print("\nUnique Values in cabin_num:")
print(df['cabin_num'].unique())

# Visualize the distribution of cabin_num
plt.figure(figsize=(10, 6))
sns.histplot(df['cabin_num'].dropna(), bins=20, kde=True, color='blue')
plt.title('Distribution of Cabin Numbers')
plt.xlabel('Cabin Number')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Define the number of buckets and labels
num_buckets = 5
labels = ['low', 'medium-low', 'medium', 'medium-high', 'high']

# Create bins based on the range of cabin_num
min_num = df['cabin_num'].min()
max_num = df['cabin_num'].max()
bins = np.linspace(min_num, max_num, num_buckets + 1)

# Bucketize cabin_num
df['cabin_num_bucket'] = pd.cut(df['cabin_num'], bins=bins, labels=labels, include_lowest=True)

# Display the updated DataFrame
print(df[['Cabin', 'cabin_num', 'cabin_num_bucket']])

# Visualize the distribution of cabin_num_bucket
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='cabin_num_bucket', order=labels, palette='viridis')
plt.title('Distribution of Cabin Number Buckets')
plt.xlabel('Cabin Number Bucket')
plt.ylabel('Count')
plt.show()

In [ ]:
#Let's plot CryoSleep against HomePlanet; Destination and Cabin

In [ ]:
df= df_filled

In [ ]:
sns.set(style="whitegrid")

# 1. CryoSleep vs HomePlanet
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x='HomePlanet', hue='CryoSleep', palette='Set2')
plt.title('CryoSleep vs HomePlanet')
plt.xlabel('HomePlanet')
plt.ylabel('Count')
plt.legend(title='CryoSleep', loc='upper right')
plt.show()

In [ ]:
# 2. CryoSleep vs Destination
# Handle NaN values in Destination by replacing them with "Unknown"
df['Destination'].fillna('Unknown', inplace=True)

plt.figure(figsize=(8, 5))
sns.countplot(data=df, x='Destination', hue='CryoSleep', palette='Set2')
plt.title('CryoSleep vs Destination')
plt.xlabel('Destination')
plt.ylabel('Count')
plt.legend(title='CryoSleep', loc='upper right')
plt.show()

In [ ]:
# 3. CryoSleep vs Cabin (split into Deck, Num, Side)
# Split the Cabin column into Deck, Num, and Side
df[['Deck', 'Num', 'Side']] = df['Cabin'].str.split('/', expand=True)

# Handle NaN values in Deck, Num, and Side by replacing them with "Unknown"
df['Deck'].fillna('Unknown', inplace=True)
df['Num'].fillna('Unknown', inplace=True)
df['Side'].fillna('Unknown', inplace=True)

# Plot CryoSleep vs Deck
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x='Deck', hue='CryoSleep', palette='Set2')
plt.title('CryoSleep vs Deck')
plt.xlabel('Deck')
plt.ylabel('Count')
plt.legend(title='CryoSleep', loc='upper right')
plt.show()

In [ ]:
# Plot CryoSleep vs Side
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x='Side', hue='CryoSleep', palette='Set2')
plt.title('CryoSleep vs Side')
plt.xlabel('Side')
plt.ylabel('Count')
plt.legend(title='CryoSleep', loc='upper right')
plt.show()

In [ ]:
# 4. CryoSleep vs Age (bucketized into age groups of 10 years)
# Create age groups
df['AgeGroup'] = pd.cut(df['Age'], bins=range(0, 101, 10), right=False, labels=[f'{i}-{i+9}' for i in range(0, 100, 10)])

# Convert AgeGroup to string type to allow adding "Unknown"
df['AgeGroup'] = df['AgeGroup'].astype(str)

# Handle NaN values in AgeGroup by replacing them with "Unknown"
df['AgeGroup'].fillna('Unknown', inplace=True)

# Plot CryoSleep vs AgeGroup
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='AgeGroup', hue='CryoSleep', palette='Set2', order=sorted(df['AgeGroup'].unique()))
plt.title('CryoSleep vs Age Group')
plt.xlabel('Age Group')
plt.ylabel('Count')
plt.legend(title='CryoSleep', loc='upper right')
plt.show()

In [ ]:
df=df_filled

In [ ]:
#Let's investigate VIP
# Create binary indicators for spending (1 if spent money, 0 if not)
spending_columns = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
for col in spending_columns:
    df[f'{col}_spent'] = df[col].apply(lambda x: 1 if x > 0 else 0)

# Melt the DataFrame for easier plotting
melted_df = df.melt(id_vars=['VIP'], value_vars=[f'{col}_spent' for col in spending_columns],
                    var_name='SpendingCategory', value_name='SpentMoney')

# Replace SpendingCategory labels for better readability
melted_df['SpendingCategory'] = melted_df['SpendingCategory'].str.replace('_spent', '')

# Plot the data
plt.figure(figsize=(12, 6))
sns.barplot(data=melted_df, x='SpendingCategory', y='SpentMoney', hue='VIP', ci=None, palette='Set2')
plt.title('Proportion of Passengers Who Spent Money by VIP Status')
plt.xlabel('Spending Category')
plt.ylabel('Proportion Who Spent Money')
plt.legend(title='VIP Status', loc='upper right')
plt.show()

In [ ]:
# Create binary indicators for spending (1 if spent money, 0 if not)
spending_columns = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
for col in spending_columns:
    df[f'{col}_spent'] = df[col].apply(lambda x: 1 if x > 0 else 0)

# Melt the DataFrame for easier plotting
melted_df = df.melt(id_vars=['VIP'], value_vars=[f'{col}_spent' for col in spending_columns],
                    var_name='SpendingCategory', value_name='SpentMoney')

# Replace SpendingCategory labels for better readability
melted_df['SpendingCategory'] = melted_df['SpendingCategory'].str.replace('_spent', '')

# Calculate the proportion of VIP statuses for each spending category
vip_proportions = melted_df.groupby(['SpendingCategory', 'SpentMoney', 'VIP']).size().unstack(fill_value=0)
vip_proportions = vip_proportions.div(vip_proportions.sum(axis=1), axis=0).reset_index()

# Filter for rows where money was spent (SpentMoney == 1)
vip_proportions = vip_proportions[vip_proportions['SpentMoney'] == 1]

# Melt the DataFrame again for plotting
vip_proportions_melted = vip_proportions.melt(id_vars=['SpendingCategory', 'SpentMoney'],
                                              value_vars=[True, False],
                                              var_name='VIP', value_name='Proportion')

# Plot the data
plt.figure(figsize=(12, 6))
sns.barplot(data=vip_proportions_melted, x='SpendingCategory', y='Proportion', hue='VIP', palette='Set2')
plt.title('Proportion of VIP Status for Passengers Who Spent Money')
plt.xlabel('Spending Category')
plt.ylabel('Proportion of VIP Status')
plt.legend(title='VIP Status', loc='upper right')
plt.show()

In [ ]:
# Melt the DataFrame for easier plotting
melted_df = df.melt(id_vars=['VIP'], value_vars=['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'],
                    var_name='SpendingCategory', value_name='AmountSpent')

# Group by VIP status and SpendingCategory, then calculate median and standard deviation
grouped_df = melted_df.groupby(['VIP', 'SpendingCategory'])['AmountSpent'].agg(['median', 'std']).reset_index()

# Plot the data
plt.figure(figsize=(12, 6))
sns.barplot(data=grouped_df, x='SpendingCategory', y='median', hue='VIP', palette='Set2', ci='sd')
plt.title('Median Amount Spent by VIP Status (with Standard Deviation Error Bars)')
plt.xlabel('Spending Category')
plt.ylabel('Median Amount Spent')
plt.legend(title='VIP Status', loc='upper right')
plt.show()

In [ ]:
cryosleep_spending = df.groupby('CryoSleep')[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum()

# Display the results
print("Total Amount Spent by CryoSleep Status:")
print(cryosleep_spending)

In [ ]:
#Hence people in CrySleep don't spend money!!!
service_columns = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

df.loc[df['CryoSleep'] == True, service_columns] = df.loc[df['CryoSleep'] == True, service_columns].fillna(0)

# Display the updated DataFrame
print("DataFrame after imputing missing values for CryoSleep passengers:")

In [ ]:
nan_rows = df[df.isna().any(axis=1)]
nan_counts = df.isna().sum()

print(nan_counts)

In [ ]:
#investigate cabins? Is there a logic per deck? 
# Check for odd/even numbering by Side
df['CabinParity'] = df['cabin_num'] % 2
df['InferredSide'] = np.where(df['CabinParity'] == 0, 'S', 'P')  # Example logic

In [ ]:
# Define the service columns
service_columns = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

# Group by Deck and calculate the average spending for each service
average_spending_by_deck = df.groupby('cabin_deck')[service_columns].median()

# Display the results
print("Median Spending by Deck Level:")
print(average_spending_by_deck)

In [ ]:
# Create a cross-tabulation of cabin_level vs cabin_side
cross_tab = pd.crosstab(df['cabin_deck'], df['cabin_side'])

# Calculate the S/P ratio for each cabin level
cross_tab['S/P Ratio'] = cross_tab['S'] / cross_tab['P']

# Display the results
print("Cross-Tabulation of Cabin Level vs Cabin Side with S/P Ratio:")
print(cross_tab)


In [ ]:
# Create a cross-tabulation of cabin_level vs cabin_side
cross_tab = pd.crosstab(df['cabin_deck'], df['VIP'])
# Display the results
print(cross_tab)